#  TensorFlow's Decision Forests (TF-DF) algorithm

Author: __Thirada Tiamklang 14337188__

        AT3 - Data Product with Machine 
        
__Table of contents__
1. Load dataset
2. Train TensorFlow's Decision Forests (TF-DF) algorithm:

        2.1 Decision Forests model
        2.2 Random forest model
        2.3 Gradient Boosted Trees

## 1. Load Dataset

In [1]:
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

In [2]:
import sys
print(sys.path)

['/Users/thiradatiamklang/Desktop/flight-streamlit-at3/flight-prediction/notebooks/TT_notebooks', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python310.zip', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/lib-dynload', '', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages']


In [3]:
import sys
sys.path.append('../../src')


In [4]:
from data.make_dataset import load_sets
X_train, y_train, X_val, y_val, X_test, y_test = load_sets(path='../../data/processed/')

In [5]:
import pandas as pd
# Read a Feather file
df = pd.read_feather('../../data/processed/df_cleaned_select_cols.feather')

In [6]:
from data.make_dataset import pop_target
features, target = pop_target(df, 'totalFare')

In [7]:
y_train

array([-0.31391783, -0.50182474, -1.22936172, ..., -0.89209292,
        1.17970114, -0.04892092])

In [8]:
X_train

array([[ 1.07916650e+07, -9.95738027e-01,  1.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 8.53247500e+06, -7.88605694e-01,  1.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.91918400e+06, -5.74075777e-01,  1.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 1.18766450e+07, -4.60646166e-01,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.16629820e+07,  4.85011571e-01,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.58366700e+06,  1.26422368e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

## 2. Train TensorFlow's Decision Forests (TF-DF)

In [11]:
import tensorflow_decision_forests as tfdf
import tensorflow as tf
from sklearn.metrics import mean_squared_error

# Convert data to tf.data.Datasets
batch_size = 100

# Assuming X_train, y_train, X_val, y_val, X_test, and y_test are NumPy arrays or tensors
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.batch(batch_size)

validation_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
validation_dataset = validation_dataset.batch(batch_size)

test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_dataset = test_dataset.batch(batch_size)


### 2.1 TensorFlow Decision Forests (TF-DF) models

In [12]:
# Define the TensorFlow Decision Forest model
model = tfdf.keras.RandomForestModel(task=tfdf.keras.Task.REGRESSION)

# Compile the model
model.compile(metrics=["mean_squared_error"])

# Fit the model
model.fit(train_dataset, epochs=1, validation_data=validation_dataset)

Use /var/folders/sb/nxrzyd0n61192x17k7wcyr4w0000gn/T/tmpxovdtnoa as temporary training directory
Reading training dataset...
Training dataset read in 0:00:09.935749. Found 8111999 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(2704000, shape=(), dtype=int32)
Validation dataset read in 0:00:03.033918. Found 2704000 examples.
Training model...


[INFO 23-11-05 17:15:41.5144 AEDT kernel.cc:1233] Loading model from path /var/folders/sb/nxrzyd0n61192x17k7wcyr4w0000gn/T/tmpxovdtnoa/model/ with prefix 50bb42066cbe42f4
[INFO 23-11-05 17:15:50.2774 AEDT decision_forest.cc:660] Model loaded with 300 root(s), 8111404 node(s), and 9 input feature(s).
[INFO 23-11-05 17:15:50.2775 AEDT abstract_model.cc:1344] Engine "RandomForestOptPred" built
[INFO 23-11-05 17:15:50.2775 AEDT kernel.cc:1061] Use fast generic engine


Model trained in 0:25:41.002621
Compiling model...
Model compiled.


In [16]:
# Evaluate the model on train set
results = model.evaluate(train_dataset)

81120/81120 [==============================] - 312s 4ms/step - loss: 0.0000e+00 - mean_squared_error: 0.3042


In [15]:
# Evaluate the model on test set
results = model.evaluate(test_dataset)

# Print evaluation results
mse = results[0]
print(f'Mean Squared Error: {mse}')

# Make predictions
predictions = model.predict(test_dataset)

27040/27040 [==============================] - 104s 4ms/step - loss: 0.0000e+00 - mean_squared_error: 0.3072
Mean Squared Error: 0.0
27040/27040 [==============================] - 105s 4ms/step


__Note:__ It can be seen that the MSE score for the train and test sets is nearly the same with low overfitting, and the MSE score is approximately `0.3072`.

In [ ]:
import tensorflow_decision_forests as tfdf
import tensorflow as tf

# Define and train the Random Forest models
model_rf1 = tfdf.keras.RandomForestModel(task=task)
model_rf2 = tfdf.keras.RandomForestModel(
    task=task, num_trees=100, max_depth=10, split_axis="SPF_OHE"
)

model_rf1.fit(train_input)
model_rf2.fit(train_input)

# Evaluate the Random Forest models
results_rf1 = model_rf1.evaluate(test_input)
results_rf2 = model_rf2.evaluate(test_input)

# Print evaluation results for Random Forest
print("Random Forest Model 1 Evaluation:")
print(results_rf1)
print("Random Forest Model 2 Evaluation:")
print(results_rf2)

# Define and train the Gradient Boosted Trees models
model_gbdt1 = tfdf.keras.GradientBoostedTreesModel(task=task)
model_gbdt2 = tfdf.keras.GradientBoostedTreesModel(
    task=task, num_trees=100, growing_strategy="BEST_FIRST_GLOBAL"
)

model_gbdt1.fit(train_input)
model_gbdt2.fit(train_input)

# Evaluate the Gradient Boosted Trees models
results_gbdt1 = model_gbdt1.evaluate(test_input)
results_gbdt2 = model_gbdt2.evaluate(test_input)

# Print evaluation results for Gradient Boosted Trees
print("Gradient Boosted Trees Model 1 Evaluation:")
print(results_gbdt1)
print("Gradient Boosted Trees Model 2 Evaluation:")
print(results_gbdt2)


### 2.3 Gradient Boosted Trees